In [2]:
from typing import Dict
from rich import print
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#mercury
import mercury as mr
#custom imports
from Load import Load_Type, Load_Category, Load, Wall, Concrete, Soil

In [3]:
#setup mercury app
# set Application parameters
app = mr.App(title="Rteain",
        description="Simple Retainfing Wall Designer.",
        show_code=False,
        show_prompt=False,
        continuous_update=False,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True,
        allow_share=True,
        stop_on_error=False
)

mercury.App

# Soil Properties

In [ ]:
#Soil Properties
gamma_s = mr.Numeric(value=0, min=0, max=1000, label="Soil Unit Weight (kPa):") # kPa/m
ka = mr.Numeric(value=0.33, min=0, max=100, label="Active Pressure S=Coefficient (ka):")
ko = mr.Numeric(value=0.0, min=0, max=1100, label="At Rest Pressure S=Coefficient (k0):")
kp = mr.Numeric(value=3.0, min=0, max=100, label="Passive Pressure S=Coefficient (kp):")
Q_uls = mr.Numeric(value=300, min=0, max=1000, label="Maximum ULS Bearing Pressure (Quls):")
Q_sls = mr.Numeric(value=200, min=0, max=1000, label="Maximum SLS Bearing Pressure (Qsls):")
mu = mr.Numeric(value=0.35, min=0, max=1, label="Friction Coefficient (mu):")

soil = Soil('soil', gamma_s.value, ka.value, ko.value, kp.value, Q_uls.value, Q_sls.value, mu.value)

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

mercury.Numeric

# Concrete Properties

In [ ]:
# concrete properties
fc = 25 # MPa
Ec = 4500*(fc**0.5) # MPa
alpha1 = 0.91
beta1 = 0.81

C25 = Concrete('C25', fc)

# Wall Geomerty

In [ ]:
# Geometry
# Wall
h = 1800 # mm
tw = 200 # mm
L = 1000 # mm


# Footing Geometry

In [ ]:

# Footing
tc = 300 # mm
toe = 250 # mm
b = 1500 # mm
D = 300 # mm
heel = b-toe-tw # mm

wall = Wall('wall1', h, tw, tc, toe, b, D, heel, C25)
wall.Cconcrete = C25

In [77]:
# Loads
# Soil
q_soil = ka*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa
q_eq = 0.25*gamma_s*((h+tc+D)/1000)*(L/1000) # kPa

# compaction
q_comp = 5 # kPa

# surcharge
W_dl = 2.4 # kPa
W_ll = 4.8 # kpa

# Wall Loads
P_dl = 100 # kN/m
P_sl = 200 # kN/m

wall.Add_Load(Load_Type.HYDROSTATIC, Load_Category.SOIL, 'soil', q_soil)
#wall.Add_Load(Load_Type.SEISMIC, Load_Category.SEISMIC, 'eq', q_eq)

wall.Add_Load(Load_Type.COMPACTION, Load_Category.COMPACTION, 'comp', q_comp)

#wall.Add_Load(Load_Type.SURCHARGE, Load_Category.DEAD, 'dl', W_dl)
#wall.Add_Load(Load_Type.SURCHARGE, Load_Category.LIVE, 'll', W_ll)

#wall.Add_Load(Load_Type.WALL, Load_Category.DEAD, 'wall_dl', P_dl)
#wall.Add_Load(Load_Type.WALL, Load_Category.SNOW, 'wall_sl', P_sl)

In [78]:
def Sum_Loads(loads: Dict, load_type: Load_Type)-> Dict:
    load_dict = {
        'Dead': 0,
        'Live': 0,
        'Storage': 0,
        'Snow': 0,
        'Seismic': 0,
        'Soil': 0,
        'Compaction': 0
    }
    for k, load in loads.items():
        if load.Type == load_type:
            if load.Category == Load_Category.DEAD:
                load_dict['Dead'] =+ load.Value
            if load.Category == Load_Category.LIVE:
                load_dict['Live'] =+ load.Value
            if load.Category == Load_Category.STORAGE_LIVE:
                load_dict['Storage'] =+ load.Value
            if load.Category == Load_Category.SNOW:
                load_dict['Snow'] =+ load.Value
            if load.Category == Load_Category.SEISMIC:
                load_dict['Seismic'] =+ load.Value
            if load.Category == Load_Category.SOIL:
                load_dict['Soil'] =+ load.Value
            if load.Category == Load_Category.COMPACTION:
                load_dict['Compaction'] =+ load.Value
    return load_dict

def SLS_Load_Combos(load_dict: Dict) -> Dict:
    combo_dict = {}
    combo_dict['1.0DL'] = load_dict['Dead'] * 1.0
    combo_dict['1.0DL+1.0LL+1.0SL+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Live'] * 1.0 + load_dict['Snow'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0H'] = load_dict['Soil']
    return combo_dict

def ULS_Load_Combos(load_dict: Dict) -> Dict:
    combo_dict = {}
    combo_dict['1.5H+1.5Compaction'] = load_dict['Soil'] * 1.5 + load_dict['Compaction'] * 1.5
    combo_dict['1.4DL'] = load_dict['Dead'] * 1.4
    combo_dict['1.25DL+1.5LL+1.0SL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 + load_dict['Snow'] * 1.0 + 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5SL+1.0LL++1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 + load_dict['Live'] * 1.0+ 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5LL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Live'] * 1.5 + 1.5 * load_dict['Soil']
    combo_dict['1.25DL+1.5SL+1.5H'] = load_dict['Dead'] * 1.25 + load_dict['Snow'] * 1.5 + 1.5 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+0.5LL+0.25SL+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + load_dict['Live'] * 0.5 + load_dict['Snow'] * 0.25 + 1.0 * load_dict['Soil']
    return combo_dict

def EQ_Load_Combos(load_dict: dict) -> Dict:
    combo_dict = {}
    combo_dict['1.0DL+1.0EQ+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + 1.0 * load_dict['Soil']
    combo_dict['1.0DL+1.0EQ+0.5LL+0.25SL+1.0H'] = load_dict['Dead'] * 1.0 + load_dict['Seismic'] * 1.0 + load_dict['Live'] * 0.5 + load_dict['Snow'] * 0.25 + 1.0 * load_dict['Soil']
    return combo_dict

In [79]:
hydro_loads = {}
sur_loads = {}
comp_loads = {}
wall_loads = {}
eq_loads = {}

hydro_loads = Sum_Loads(wall.Loads, Load_Type.HYDROSTATIC)
sur_loads = Sum_Loads(wall.Loads, Load_Type.SURCHARGE)
comp_loads = Sum_Loads(wall.Loads, Load_Type.COMPACTION)
wall_loads = Sum_Loads(wall.Loads, Load_Type.WALL)
eq_loads = Sum_Loads(wall.Loads, Load_Type.SEISMIC)

loads = {
    'hydro': hydro_loads,
    'surcharge': sur_loads,
    'compaction': comp_loads,
    'wall': wall_loads,
    'seismic': eq_loads
}

df_loads = pd.DataFrame(loads)
df_loads

,hydro,surcharge,compaction,wall,seismic
Dead,0.000,0,0,0,0
Live,0.000,0,0,0,0
Storage,0.000,0,0,0,0
Snow,0.000,0,0,0,0
Seismic,0.000,0,0,0,0
Soil,15.048,0,0,0,0
Compaction,0.000,0,5,0,0


In [80]:
hydro_uls_combos = {}
sur_uls_combos = {}
comp_uls_combos = {}
wall_uls_combos = {}
eq_combos = {}

hydro_uls_combos = ULS_Load_Combos(hydro_loads)
sur_uls_combos = ULS_Load_Combos(sur_loads)
comp_uls_combos = ULS_Load_Combos(comp_loads)
wall_uls_combos = ULS_Load_Combos(wall_loads)
eq_combos = EQ_Load_Combos(eq_loads)

hydro_sls_combos = {}
sur_sls_combos = {}
wall_sls_combos = {}

hydro_sls_combos = SLS_Load_Combos(hydro_loads)
sur_sls_combos = SLS_Load_Combos(sur_loads)
wall_sls_combos = SLS_Load_Combos(wall_loads)

for k, v in eq_combos.items():
    pass
    #print(f"{k} -> {v}")

In [81]:
def Hydro(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    heel = wall.Heel_Length/1000
    P = load*h/2
    x = h/3
    M = P*x
    W = load * heel * h
    return {'P': P, 'x': x, 'M': M, 'W': W}

def Surcharge(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    heel = wall.Heel_Length/1000
    P = load*h
    x = h/2
    M = P*x
    W = load * heel * h
    return {'P': P, 'x': x, 'M': M, 'W': W}

def Compaction(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    P = load*h
    x = h/2
    M = P*x
    W = 0
    return {'P': P, 'x': x, 'M': M, 'W': W}

def Wall(wall: Wall, load: float):
    P = load
    x = (wall.Wall_Thickness/2 + wall.Toe_length)/1000
    M = P*x
    W = P
    return {'P': P, 'x': x, 'M': M, 'W': W}

def Seismic(wall: Wall, load: float):
    h = (wall.Height + wall.Toe_Cover + wall.Footing_Thickness)/1000
    P = load*h/2
    x = 2*h/3
    M = P*x
    W = 0
    return {'P': P, 'x': x, 'M': M, 'W': W}

def Analyze_Wall(load_type: Load_Type, wall: Wall, combos: Dict):
    resultants = {}

    for combo, load in combos.items():
        if load_type == Load_Type.HYDROSTATIC:
            resultants[combo] = Hydro(wall, load)
        if load_type == Load_Type.COMPACTION:
            resultants[combo] = Compaction(wall, load)
        if load_type == Load_Type.SURCHARGE:
            resultants[combo] = Surcharge(wall, load)
        if load_type == Load_Type.WALL:
            resultants[combo] = Wall(wall, load)
        if load_type == Load_Type.SEISMIC:
            resultants[combo] = Seismic(wall, load)
    
    return resultants


In [82]:
hydro_sls_res = Analyze_Wall(Load_Type.HYDROSTATIC, wall, hydro_sls_combos)
sur_sls_res = Analyze_Wall(Load_Type.SURCHARGE, wall, sur_sls_combos)
wall_sls_res = Analyze_Wall(Load_Type.WALL, wall, wall_sls_combos)
eq_res = Analyze_Wall(Load_Type.SEISMIC, wall, eq_combos)

In [83]:
hydro_uls_res = Analyze_Wall(Load_Type.HYDROSTATIC, wall, hydro_uls_combos)
comp_uls_res = Analyze_Wall(Load_Type.COMPACTION, wall, comp_uls_combos)
sur_uls_res = Analyze_Wall(Load_Type.SURCHARGE, wall, sur_uls_combos)
wall_uls_res = Analyze_Wall(Load_Type.WALL, wall, wall_uls_combos)

In [84]:
geo_loads = {}
struct_loads = {}

def Sum_Combos(*combos: Dict):
    combined_dict = {}
    for d in combos:
        for key, nested_dict in d.items():
                if key not in combined_dict:
                    combined_dict[key] = {}
                for sub_key, value in nested_dict.items():
                    if sub_key == 'x':  # Skip the 'x' key
                        continue
                    if sub_key in combined_dict[key]:
                        combined_dict[key][sub_key] += value
                    else:
                        combined_dict[key][sub_key] = value
    return dict(combined_dict)

In [85]:
geo_loads = Sum_Combos(hydro_sls_res, sur_sls_res, wall_sls_res)
geo_df = pd.DataFrame(geo_loads).transpose()
geo_df

,P,M,W
1.0DL,0.0000,0.00000,0.00000
1.0DL+1.0LL+1.0SL+1.0H,18.0576,14.44608,37.92096
1.0H,18.0576,14.44608,37.92096


In [86]:
struct_loads = Sum_Combos(hydro_uls_res, comp_uls_res, sur_uls_res, wall_uls_res, eq_res)
struct_df = pd.DataFrame(struct_loads).transpose()
struct_df

,P,M,W
1.5H+1.5Compaction,45.0864,43.26912,56.88144
1.4DL,0.0000,0.00000,0.00000
1.25DL+1.5LL+1.0SL+1.5H,27.0864,21.66912,56.88144
1.25DL+1.5SL+1.0LL++1.5H,27.0864,21.66912,56.88144
1.25DL+1.5LL+1.5H,27.0864,21.66912,56.88144
1.25DL+1.5SL+1.5H,27.0864,21.66912,56.88144
1.0DL+1.0EQ+1.0H,18.0576,14.44608,37.92096
1.0DL+1.0EQ+0.5LL+0.25SL+1.0H,18.0576,14.44608,37.92096


In [87]:
w_wall = C25.Unit_Weight * wall.Height * wall.Wall_Thickness / 1000**2
w_ftg = C25.Unit_Weight* wall.Footing_Thickness * wall.Footing_Length / 1000**2
#w_soil = soil.Density * wall.Heel_Length * wall.Height / 1000**2
w_toe_soil = soil.Density * wall.Toe_length * wall.Toe_Cover / 1000**2

In [88]:
#check sliding
sliding_results = {}

f_res = (w_wall + w_ftg + w_toe_soil) 

for k, combo in geo_loads.items():
    P = combo['P']
    Pr = (f_res + hydro_sls_res[k]['W'] + wall_sls_res[k]['W']) * soil.Mu
    ratio = P/Pr
    sliding_results[k] = {'P': P, 'Pr': Pr, 'Ratio': ratio} 

df = pd.DataFrame(sliding_results).transpose()
df

,P,Pr,Ratio
1.0DL,0.0000,8.346000,0.000000
1.0DL+1.0LL+1.0SL+1.0H,18.0576,23.514384,0.767938
1.0H,18.0576,23.514384,0.767938


In [95]:
#check overturning
ot_results ={}

e_toe = 0.5 * wall.Toe_length / 1000
m_toe = w_toe_soil * e_toe

e_wall = (0.5 * wall.Wall_Thickness + wall.Toe_length) / 1000
m_wall = w_wall * e_wall

e_ftg = 0.5 * wall.Footing_Length / 1000
m_ftg = w_ftg * e_ftg

e_soil = (0.5 * wall.Heel_Length + wall.Wall_Thickness + wall.Toe_length) / 1000

for k, combo in geo_loads.items():
    M_ot = combo['M']
    M_rt = m_toe + m_wall + m_ftg + hydro_sls_res[k]['W'] * e_soil + wall_sls_res[k]['W'] * e_wall
    ratio = M_ot/M_rt
    ot_results[k] = {"Mot": M_ot, "Mrt": M_rt, "Ratio": ratio}

df = pd.DataFrame(ot_results).transpose()
df

,Mot,Mrt,Ratio
1.0DL,0.00000,11.302125,0.000000
1.0DL+1.0LL+1.0SL+1.0H,14.44608,48.275061,0.299245
1.0H,14.44608,48.275061,0.299245


In [110]:
#check bearing
brg_results = {}

for k, combo in struct_loads.items():
    location = 'inside'
    if k == '1.5H+1.5Compaction': 
        continue
    
    M_ot = combo['M']
    M_rt = m_toe + m_wall + hydro_uls_res[k]['W'] * e_soil + wall_uls_res[k]['W'] * e_wall
    Mnet = abs(M_ot - M_rt)

    P = combo['P'] + w_wall + w_ftg + w_toe_soil

    d = (Mnet / P) * 1000
    e = wall.Footing_Length / 2 - D

    #print(f"Mot: {M_ot} - M_rt: {M_rt} = Mnet: {Mnet} P: {P} d: {d}")

    if e > wall.Footing_Length / 6:
        location = 'outside'

    if location == 'inside':
        Qmax = P/((wall.Footing_Length / 1000) * 1) + M_ot * ((wall.Footing_Length / 1000) / 2) / ((wall.Footing_Thickness / 1000) * (wall.Footing_Length / 1000)**3 / 12)
        Qmin = P/((wall.Footing_Length / 1000) * 1) - M_ot * ((wall.Footing_Length / 1000) / 2) / ((wall.Footing_Thickness / 1000) * (wall.Footing_Length / 1000)**3 / 12)
    else:
        Qmax = (2 * P) / (3 * (d / 1000) * (wall.Footing_Length / 1000))
        Qmin = 0.0

    ratio = Qmax / soil.Q_uls

    brg_results[k] = {'Qmax': Qmax, 'Qmin': Qmin, 'Q_uls': soil.Q_uls, 'Ratio': ratio, 'e': e, 'Location': location}

df = pd.DataFrame(brg_results).transpose()
df

,Qmax,Qmin,Q_uls,Ratio,e,Location
1.4DL,60.424905,0.0,300,0.201416,450.0,outside
1.25DL+1.5LL+1.0SL+1.5H,27.625328,0.0,300,0.092084,450.0,outside
1.25DL+1.5SL+1.0LL++1.5H,27.625328,0.0,300,0.092084,450.0,outside
1.25DL+1.5LL+1.5H,27.625328,0.0,300,0.092084,450.0,outside
1.25DL+1.5SL+1.5H,27.625328,0.0,300,0.092084,450.0,outside
1.0DL+1.0EQ+1.0H,26.16969,0.0,300,0.087232,450.0,outside
1.0DL+1.0EQ+0.5LL+0.25SL+1.0H,26.16969,0.0,300,0.087232,450.0,outside
